# 카메라 스티커앱을 개선하자!

## 프로젝트 (1) moviepy로 비디오 처리하기

### 1. `moviepy`를 이용해서 주피터 노트북상에서 비디오를 읽고 쓰는 프로그램을 작성해봅시다.

In [1]:
from moviepy.editor import VideoClip, VideoFileClip
from moviepy.editor import ipython_display
import cv2
import numpy as np
import os

In [2]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# 쓰기
result_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
clip.write_videofile(result_video_path)

t:   0%|          | 0/404 [00:00<?, ?it/s, now=None]                

Moviepy - Building video /home/aiffel0039/aiffel/video_sticker_app/images/mvpyresult.mp4.
MoviePy - Writing audio in mvpyresultTEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video /home/aiffel0039/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel0039/aiffel/video_sticker_app/images/mvpyresult.mp4


### 2. `moviepy`로 읽은 동영상을 `numpy` 형태로 변환하고 영상 밝기를 50% 어둡게 만든 후에 저장해주세요.

In [3]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# clip 에서 numpy 로 데이터 추출
vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)
for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

# 새 clip 만들기
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

# 쓰기
result_video_path2 = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult2.mp4'
outclip.write_videofile(result_video_path2, fps=30)

t:  15%|█▍        | 59/403 [00:00<00:00, 587.18it/s, now=None]

Moviepy - Building video /home/aiffel0039/aiffel/video_sticker_app/images/mvpyresult2.mp4.
Moviepy - Writing video /home/aiffel0039/aiffel/video_sticker_app/images/mvpyresult2.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel0039/aiffel/video_sticker_app/images/mvpyresult2.mp4


### 3. 영상을 읽고 쓰는 시간을 측정해주세요. `openCV`를 사용할 때와 차이를 측정해보세요.

In [4]:
# CASE 1 : moviepy 사용
start = cv2.getTickCount()
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)

vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)

for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

mvpy_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
outclip.write_videofile(mvpy_video_path, fps=30)

time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] moviepy time : {time:.2f}ms')

t:  14%|█▍        | 56/403 [00:00<00:00, 552.87it/s, now=None]

Moviepy - Building video /home/aiffel0039/aiffel/video_sticker_app/images/mvpyresult.mp4.
Moviepy - Writing video /home/aiffel0039/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel0039/aiffel/video_sticker_app/images/mvpyresult.mp4
[INFO] moviepy time : 2.62ms


In [5]:
# CASE 2 : OpenCV 사용
start = cv2.getTickCount()
vc = cv2.VideoCapture(video_path)

cv_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/cvresult.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter(cv_video_path, fourcc, 30, (640,360))

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False: break
    
    img_result = cv2.resize(img, (640, 360)) * 0.5
    vw.write(img_result.astype(np.uint8))
    
time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] cv time : {time:.2f}ms')

[INFO] cv time : 1.29ms


### 4. `moivepy`를 이용할 때의 장단점을 분석해봅시다. 


- moviepy는 동영상을 읽는 것은 쉽지만 numpy 데이터로 변환을 해줘야하는 과정이 있습니다.
- 또한 프레임 이미지에 검출 모델을 적용하더라도 결과 동영상을 확인하기 위해서는 동영상 파일을 저장했다가 다시 읽어야 합니다. 
- moviepy를 사용하면 opencv를 사용할 때 보다 시간이 더 소요되는 것을 확인할 수 있습니다.


## 프로젝트 (2) 어디까지 만들고 싶은지 정의하기

### 1. 웹캠을 이용한 실시간 스티커 앱을 만들어주세요

지난 시간에 우리는 `img2sticker_orig`라는 이름의 메소드를 구현하여 `addsticker.py`에 저장하였습니다. 해당 메소드를 그대로 복사하여 `img2sticker`라는 이름의 메소드를 만들고 이를 `newaddsticker.py`에 저장하겠습니다. 오늘의 프로젝트는 이 메소드를 보완하여 구현하는 방식으로 진행될 것입니다.

`cv2.VideoCapture(0)`를 이용하면 웹캠 입력을 받아올 수 있습니다. 현재의 기본적인 `img2sticker` 를 활용하여 가장 기본적인 웹캠 실시간 스티커앱을 아래와 같이 빠르게 만들어 보겠습니다.